# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine
import re
import numpy as np

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer, WordNetLemmatizer

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier

import nltk
nltk.download('wordnet')


# suppress warnings from final output
import warnings
warnings.simplefilter("ignore")

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\xjuan\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
def load_data():
    # load data from database
    engine = create_engine('sqlite:///DisasterResponse.db')
    df = pd.read_sql("SELECT * FROM msg_df", con=engine)
    X = df['message']
    y = df.iloc[:, 4:]
    
    return X, y, df

In [3]:
X, y, df = load_data()

### 2. Write a tokenization function to process your text data

In [4]:
def tokenize(text):
    """Method to tokenize and stem text
    
    Args:
    text: str
    
    Returns:
    text: str - tokenized
    """
    
    # normalization
    text = re.sub(r"[^a-zA-Z0-9]", " ", text)
    
    # tokenization
    token_words = word_tokenize(text.lower())
    
    # remove stopwords
    stop_words = set(stopwords.words('english'))
    filtered_text = [w for w in token_words if not w in stop_words]
    
    # stemming and lemmatization
    stemmer = SnowballStemmer('english')
    lemmatizer = WordNetLemmatizer()
    
    stemmed_text = []
    for word in filtered_text:
        stemmed = stemmer.stem(word)
        lemmatized = lemmatizer.lemmatize(stemmed).lower().strip()
        stemmed_text.append(lemmatized)
    
    return stemmed_text

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [5]:
# create random forest classifier pipeline
pipeline_rfc = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(estimator = RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [6]:
# creat traing, test sets
X_train, X_test, y_train, y_test = train_test_split(X, y)

# fit model
pipeline_rfc.fit(X_train, y_train)
y_pred = pipeline_rfc.predict(X_test)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [7]:
def display_results(y_pred, y_test):
    """Method to iterate classification report per target and overall accuracy
    
    Args:
    y_pred: panda series
    y_test: panda series
    
    Returns:
    None
    """
    # calculated overall accuracy
    print('Overall accuracy of current model is {:.3f}'.format((y_pred == y_test).mean().mean()))
    
    # classification report per column
    col_names = df.columns[4:]
    y_pred_df = pd.DataFrame(y_pred, columns=col_names)
    for col in y_pred_df.columns:
        results = classification_report(y_test[col], y_pred_df[col])
        print('Feature:', col, '\n', results)

In [8]:
display_results(y_pred, y_test)

Overall accuracy of current model is 0.948
Feature: related 
               precision    recall  f1-score   support

           0       0.70      0.41      0.51      1526
           1       0.84      0.94      0.89      4980
           2       0.43      0.40      0.41        48

    accuracy                           0.81      6554
   macro avg       0.66      0.58      0.60      6554
weighted avg       0.80      0.81      0.80      6554

Feature: request 
               precision    recall  f1-score   support

           0       0.90      0.98      0.94      5436
           1       0.83      0.48      0.61      1118

    accuracy                           0.89      6554
   macro avg       0.87      0.73      0.77      6554
weighted avg       0.89      0.89      0.88      6554

Feature: offer 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      6532
           1       0.00      0.00      0.00        22

    accuracy                  

Feature: direct_report 
               precision    recall  f1-score   support

           0       0.86      0.98      0.92      5290
           1       0.79      0.36      0.49      1264

    accuracy                           0.86      6554
   macro avg       0.83      0.67      0.71      6554
weighted avg       0.85      0.86      0.84      6554



### 6. Improve your model
Use grid search to find better parameters. 

In [9]:
pipeline_rfc.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(tokenizer=<function tokenize at 0x000001890FFB21F8>)),
  ('tfidf', TfidfTransformer()),
  ('clf', MultiOutputClassifier(estimator=RandomForestClassifier()))],
 'verbose': False,
 'vect': CountVectorizer(tokenizer=<function tokenize at 0x000001890FFB21F8>),
 'tfidf': TfidfTransformer(),
 'clf': MultiOutputClassifier(estimator=RandomForestClassifier()),
 'vect__analyzer': 'word',
 'vect__binary': False,
 'vect__decode_error': 'strict',
 'vect__dtype': numpy.int64,
 'vect__encoding': 'utf-8',
 'vect__input': 'content',
 'vect__lowercase': True,
 'vect__max_df': 1.0,
 'vect__max_features': None,
 'vect__min_df': 1,
 'vect__ngram_range': (1, 1),
 'vect__preprocessor': None,
 'vect__stop_words': None,
 'vect__strip_accents': None,
 'vect__token_pattern': '(?u)\\b\\w\\w+\\b',
 'vect__tokenizer': <function __main__.tokenize(text)>,
 'vect__vocabulary': None,
 'tfidf__norm': 'l2',
 'tfidf__smooth_idf': True,
 'tfidf__sublinear_tf': False,


In [10]:
# set parameters
parameters = {'clf__estimator__n_estimators': [10, 20],
             'clf__estimator__criterion': ['entropy']}

# fit model
cv = GridSearchCV(pipeline_rfc, parameters)
cv.fit(X_train, y_train)

# check the best estimator
cv.best_params_

{'clf__estimator__criterion': 'entropy', 'clf__estimator__n_estimators': 20}

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [12]:
# yield predicted y and display results
y_pred = cv.predict(X_test)
display_results(y_pred, y_test)

Overall accuracy of current model is 0.945
Feature: related 
               precision    recall  f1-score   support

           0       0.68      0.44      0.53      1526
           1       0.84      0.94      0.89      4980
           2       0.41      0.33      0.37        48

    accuracy                           0.81      6554
   macro avg       0.64      0.57      0.60      6554
weighted avg       0.80      0.81      0.80      6554

Feature: request 
               precision    recall  f1-score   support

           0       0.90      0.98      0.94      5436
           1       0.82      0.46      0.59      1118

    accuracy                           0.89      6554
   macro avg       0.86      0.72      0.76      6554
weighted avg       0.88      0.89      0.88      6554

Feature: offer 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      6532
           1       0.00      0.00      0.00        22

    accuracy                  

Feature: other_weather 
               precision    recall  f1-score   support

           0       0.95      1.00      0.97      6192
           1       0.82      0.02      0.05       362

    accuracy                           0.95      6554
   macro avg       0.88      0.51      0.51      6554
weighted avg       0.94      0.95      0.92      6554

Feature: direct_report 
               precision    recall  f1-score   support

           0       0.86      0.97      0.92      5290
           1       0.77      0.36      0.49      1264

    accuracy                           0.85      6554
   macro avg       0.82      0.66      0.70      6554
weighted avg       0.84      0.85      0.83      6554



### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [13]:
# create kneighbors classifier
pipeline_knc = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(estimator = KNeighborsClassifier()))
])

pipeline_knc.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(tokenizer=<function tokenize at 0x000001890FFB21F8>)),
  ('tfidf', TfidfTransformer()),
  ('clf', MultiOutputClassifier(estimator=KNeighborsClassifier()))],
 'verbose': False,
 'vect': CountVectorizer(tokenizer=<function tokenize at 0x000001890FFB21F8>),
 'tfidf': TfidfTransformer(),
 'clf': MultiOutputClassifier(estimator=KNeighborsClassifier()),
 'vect__analyzer': 'word',
 'vect__binary': False,
 'vect__decode_error': 'strict',
 'vect__dtype': numpy.int64,
 'vect__encoding': 'utf-8',
 'vect__input': 'content',
 'vect__lowercase': True,
 'vect__max_df': 1.0,
 'vect__max_features': None,
 'vect__min_df': 1,
 'vect__ngram_range': (1, 1),
 'vect__preprocessor': None,
 'vect__stop_words': None,
 'vect__strip_accents': None,
 'vect__token_pattern': '(?u)\\b\\w\\w+\\b',
 'vect__tokenizer': <function __main__.tokenize(text)>,
 'vect__vocabulary': None,
 'tfidf__norm': 'l2',
 'tfidf__smooth_idf': True,
 'tfidf__sublinear_tf': False,
 'tf

In [14]:
parameters_knc = {'clf__estimator__n_neighbors':[3, 5],
             'clf__estimator__leaf_size': [10, 30]}
cv_knc = GridSearchCV(pipeline_knc, param_grid = parameters_knc)
cv_knc.fit(X_train, y_train)
y_pred_knc = cv_knc.predict(X_test)

In [15]:
display_results(y_pred_knc, y_test)

Overall accuracy of current model is 0.929
Feature: related 
               precision    recall  f1-score   support

           0       0.58      0.32      0.41      1526
           1       0.81      0.92      0.86      4980
           2       0.26      0.35      0.30        48

    accuracy                           0.78      6554
   macro avg       0.55      0.53      0.53      6554
weighted avg       0.75      0.78      0.75      6554

Feature: request 
               precision    recall  f1-score   support

           0       0.84      0.99      0.91      5436
           1       0.76      0.09      0.17      1118

    accuracy                           0.84      6554
   macro avg       0.80      0.54      0.54      6554
weighted avg       0.83      0.84      0.78      6554

Feature: offer 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      6532
           1       0.00      0.00      0.00        22

    accuracy                  

Feature: shops 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      6528
           1       0.00      0.00      0.00        26

    accuracy                           1.00      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       0.99      1.00      0.99      6554

Feature: aid_centers 
               precision    recall  f1-score   support

           0       0.99      1.00      0.99      6460
           1       0.00      0.00      0.00        94

    accuracy                           0.99      6554
   macro avg       0.49      0.50      0.50      6554
weighted avg       0.97      0.99      0.98      6554

Feature: other_infrastructure 
               precision    recall  f1-score   support

           0       0.96      1.00      0.98      6286
           1       1.00      0.00      0.01       268

    accuracy                           0.96      6554
   macro avg       0.98      0.50      0.49      6554
we

### 9. Export your model as a pickle file

In [16]:
import pickle
pickle_file = 'disaster_model.pkl'

with open(pickle_file, 'wb') as file:
    pickle.dump(cv, file)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.